In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# import "data/raw/new_clean_data_all.xlsx" as dataframe
df = pd.read_excel("data/raw/new_clean_data_all.xlsx")
label_list = ["無標註", "自殺與憂鬱", "無助或無望", "正向文字", "其他負向文字", "生理反應或醫療狀況","自殺行為"]
other_list = ["無助或無望", "正向文字", "其他負向文字", "生理反應或醫療狀況"]
# label_list = ["無標註", "自殺與憂鬱","自殺行為","其他類型"]

# set all of df['其他類型'] = 0 
df['其他類型'] = 0

for idx, row in df.iterrows():
    for label in label_list:
        if row[label] == 1:
            if label in other_list:
                df.loc[idx, '其他類型'] = 1
            break
print(df.head())
# df.to_excel("data/raw/new_clean_data_all.xlsx", index=False)






   Unnamed: 0  No.      TextID     Title  \
0           0    1  0038f0569e  憂鬱症的我...   
1           1    2  0038f0569e  憂鬱症的我...   
2           2    3  0038f0569e  憂鬱症的我...   
3           3    4  0038f0569e  憂鬱症的我...   
4           4    5  0038f0569e  憂鬱症的我...   

                                 Sentence  無標註  自殺與憂鬱  無助或無望  正向文字  其他負向文字  \
0  以下文長...如果排版上有問題請見諒:loudly_crying_face:    1      0      0     0       0   
1                  其實從小到大，我自認為我自己的個性就是樂天派    0      0      0     1       0   
2             給人的感覺就是瘋瘋的，但是要認真的時候我也是會很認真的    1      0      0     0       0   
3                         而且我又是個喜歡到處出去玩的人    0      0      0     1       0   
4                     認識朋友、運動、跳舞都是我的興趣和樂趣    0      0      0     1       0   

   生理反應或醫療狀況  自殺行為  其他類型  
0          0     0     0  
1          0     0     1  
2          0     0     0  
3          0     0     1  
4          0     0     1  


In [6]:
df = pd.read_excel("data/raw/new_clean_data_all.xlsx")
four_label_list = ["無標註", "自殺與憂鬱","自殺行為", "其他類型"]
df_list = []
df_list.append(df[df['無標註'] == 1])
df_list.append(df[df['自殺與憂鬱'] == 1])
df_list.append(df[df['自殺行為'] == 1])
df_list.append(df[df['其他類型'] == 1])


In [9]:
target_df = []
test_df = []
for a_df in df_list:
    # sample 120 data from each df
    target_df.append(a_df.sample(n=120, random_state=1))
    # get rest of data from each df
    remain_df = a_df.drop(target_df[-1].index)
    test_df.append(remain_df.sample(n=19, random_state=1))
# concat all of target_df
partial_df = pd.concat(target_df,axis=0, ignore_index=True)
all_test_df = pd.concat(test_df,axis=0, ignore_index=True)
partial_df.to_excel("data/raw/new_clean_data_partial_120.xlsx", index=False)
all_test_df.to_excel("data/raw/new_clean_data_partial_120_test.xlsx", index=False)

In [2]:
import pandas as pd
import numpy as np
from pprint import pprint
import random
# import "data/raw/four_class_augmented_0526_v2.xlsx" as pandas dataframe
df = pd.read_excel("data/raw/new_clean_data_partial_120_test.xlsx")
# df = pd.read_excel("data/raw/four_class_augmented_0526_v2.xlsx")
df.head()
# construct a list of dict, with small dict has key "Sentence" and "label"
# the label is one of ["無標註", "自殺與憂鬱", "自殺行為", "其他類型"]
# the sentence is the "Sentence" columns in the excel file
label_list = ["無標註", "自殺與憂鬱", "自殺行為", "其他類型"]
# description = ["情緒為中性的句子", "自殺與憂鬱情緒句子", "有自殺行為的句子", "其他類型的句子"]
description = label_list
sentences_list = []
max_len = 0 
for idx, row in df.iterrows():
    sentence_dict = {}
    sentence_dict["content"] = str(row["Sentence"])
    if len(sentence_dict["content"]) > max_len:
        max_len = len(sentence_dict["content"])
    for id, label in enumerate(label_list):
        if row[label] == 1:
            sentence_dict["summary"] = description[id].encode('utf-8', errors='ignore').decode('utf-8')
            break
    sentences_list.append(sentence_dict)
print(max_len)
# Shuffle the data

random.seed(1)
random.shuffle(sentences_list)
pprint(sentences_list[:5])



    

30
[{'content': '甚至疏遠妳', 'summary': '自殺與憂鬱'},
 {'content': '一直痛到連腰部也會痛，陰道也有異常的出血（不像月經的血）', 'summary': '其他類型'},
 {'content': '我自己也跟閨蜜說 復學不是為了學業或是證書 而是為了證明自己', 'summary': '無標註'},
 {'content': '會再以傷害自己的方式懲罰自己。', 'summary': '自殺行為'},
 {'content': '只要遲到那一年就沒得出門，', 'summary': '無標註'}]


In [4]:
# output the data to csv file
import csv
with open('ChatGLM/ptuning_v2/new_clean_data_partial_120_test.csv', 'w', newline='', encoding='utf-8') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)  
    # writing the fields  
    csvwriter.writerow(["content", "summary"])  
    # writing the data rows  
    csvwriter.writerows([[sentence["content"], sentence["summary"]] for sentence in sentences_list])




In [2]:
# write the list of dict to a json file
import json
with open("ChatGLM/ptuning_v2/Sentences_data/test_sentences_120.json", "w",  encoding='utf-8') as f:
    for dict in sentences_list:
        # 使用 json.dump() 方法將 dict 寫入到檔案中
        # dict['summary'] = dict['summary'].encode('utf-8', errors='ignore').decode('utf-8')
        json.dump(dict, f,  ensure_ascii=False)
        # 寫入一個換行符來區分不同的字典
        f.write("\n")
# with open("./train_sentences.json", "w", encoding='utf-8') as f:
#     json.dump(sentences_list, f)
